# Notebook 2: MED-PC Calculating Latencies

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from medpc2excel.medpc_read import medpc_read

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

In [5]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (13.5,7.5)

# Loading in Recording and Metadata from Previous Notebook

# NOTE: If you changed the directory of where the MED-PC recording dataframes, then you must change the directories in the following paths

- This will get all the files in each specified path. The `*` is called a wildcard, and it can be replaced with any number of characters.
    - For more information: https://www.malikbrowne.com/blog/a-beginners-guide-glob-patterns

In [6]:
cohort = "rce_training"

In [7]:
concatted_medpc_files = glob.glob("./proc/extracted_recording_data_and_metadata/rce_training_20240215_20240305/MEDPC_recording_metadata_rce_training_20240215_20240305.csv")
metadata_files = glob.glob("./proc/extracted_recording_data_and_metadata/rce_training_20240215_20240305/MEDPC_recording_metadata_rce_training_20240215_20240305.csv")
recording_and_metadata_files = glob.glob("./proc/extracted_recording_data_and_metadata/rce_training_20240215_20240305/MEDPC_recording_metadata_rce_training_20240215_20240305.csv")

# NOTE: If there is more than one MED-PC recording dataframes, then you must manually change the path in the `pd.read_csv()` for the corresponding variables. Verify if this is the correct file that you want to use

In [8]:
metadata_files[0]

'./proc/extracted_recording_data_and_metadata/rce_training_20240215_20240305/MEDPC_recording_metadata_rce_training_20240215_20240305.csv'

In [9]:
concatted_medpc_files[0]

'./proc/extracted_recording_data_and_metadata/rce_training_20240215_20240305/MEDPC_recording_metadata_rce_training_20240215_20240305.csv'

In [10]:
recording_and_metadata_files[0]

'./proc/extracted_recording_data_and_metadata/rce_training_20240215_20240305/MEDPC_recording_metadata_rce_training_20240215_20240305.csv'

In [11]:
metadata_df = pd.read_csv(metadata_files[0], index_col=0)
concatted_medpc_df = pd.read_csv(concatted_medpc_files[0], index_col=0)
recording_and_metadata_df = pd.read_csv(recording_and_metadata_files[0], index_col=0)

- The Dataframe the contains the metadata(columns) of each recording file(rows)
    - We will mostly use this to get the cage number for each subject

In [12]:
metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,85.25,64.0,399.0,0.0,60.01,86.12,1.0,7.0,54.0,13.0,32.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
1,86.29,144.0,399.0,0.0,140.01,86.40,1.0,27.0,55.0,13.0,33.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
2,86.63,234.0,399.0,0.0,230.01,86.65,1.0,57.0,56.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
3,86.72,314.0,399.0,0.0,310.01,86.74,1.0,17.0,58.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
4,86.83,389.0,399.0,0.0,385.01,86.86,1.0,32.0,59.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt


- The Dataframe the contains the recording data for all the files.
    - The 1st row is the first data entries for each type of data(with the type being specified by the column label). This does not mean that all these are related or occured for the same trial. i.e. There can be many port entries and exits for a subject before and after a tone is played.
    - Each recording session will usually have a few thousand rows(although, most of of the columns will probably be blank by the end). And then after the last row of one session, the next row will be the next session with a different `file_path`

In [13]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,85.25,64.0,399.0,0.0,60.01,86.12,1.0,7.0,54.0,13.0,32.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
1,86.29,144.0,399.0,0.0,140.01,86.40,1.0,27.0,55.0,13.0,33.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
2,86.63,234.0,399.0,0.0,230.01,86.65,1.0,57.0,56.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
3,86.72,314.0,399.0,0.0,310.01,86.74,1.0,17.0,58.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
4,86.83,389.0,399.0,0.0,385.01,86.86,1.0,32.0,59.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt


- This dataframe combines the recording data and metadata dataframes. This is done by using the subject ID as a common column to merge together off of.

In [14]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,85.25,64.0,399.0,0.0,60.01,86.12,1.0,7.0,54.0,13.0,32.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
1,86.29,144.0,399.0,0.0,140.01,86.40,1.0,27.0,55.0,13.0,33.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
2,86.63,234.0,399.0,0.0,230.01,86.65,1.0,57.0,56.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
3,86.72,314.0,399.0,0.0,310.01,86.74,1.0,17.0,58.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
4,86.83,389.0,399.0,0.0,385.01,86.86,1.0,32.0,59.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt


In [15]:
recording_and_metadata_df["subject"] = recording_and_metadata_df["subject"].astype(str)

# Getting the Cage Numbers and the Dates for the file names

# Adjusting the days and cages that are being looked at

- List of all the subject names

In [16]:
all_subjects = sorted([str(subject_id) for subject_id in recording_and_metadata_df["subject"].unique()])

In [17]:
all_subjects

['RCE3_1',
 'RCE3_3',
 'RCE3_4',
 'RCE4_2',
 'RCE4_3',
 'RCE4_4',
 'RCE5_2',
 'RCE5_3',
 'RCE5_4']

- List of all the dates

In [18]:
all_dates = sorted(recording_and_metadata_df["date"].unique())

In [19]:
all_dates

[np.int64(20240215),
 np.int64(20240216),
 np.int64(20240217),
 np.int64(20240218),
 np.int64(20240219),
 np.int64(20240220),
 np.int64(20240221),
 np.int64(20240222),
 np.int64(20240223),
 np.int64(20240224),
 np.int64(20240225),
 np.int64(20240226),
 np.int64(20240227),
 np.int64(20240228),
 np.int64(20240229),
 np.int64(20240301),
 np.int64(20240302),
 np.int64(20240304),
 np.int64(20240305)]

# NOTE: Follow directions below if you want to change which cages or dates to plot

- You must change the cell below by clicking on the cell and then pressing the `esc` key. Then press the `Y` button on your keyboard. If you want to switch it back do the same but press the `R` button instead. This switches it from a cell that is run as if it's code, or back to a cell where it's just treated as text
- Then from above, copy the numbers that you want to use for the cage numbers and the dates

In [20]:
# recording_and_metadata_df = recording_and_metadata_df[recording_and_metadata_df["cage"].isin(all_cages) & recording_and_metadata_df["date"].isin(all_dates)]

# Getting the Cage Numbers and the Dates to name the files

In [21]:
# Getting the first and last recording date to get a range
earliest_date = min(all_dates)
latest_date = max(all_dates)

In [22]:
earliest_date

np.int64(20240215)

In [23]:
latest_date

np.int64(20240305)

# Getting the times that the tones are being played

- Getting all the rows that correspond to one of the files paths. This will be the equivalent of getting all the rows for one session

In [24]:
example_file = recording_and_metadata_df["file_path"].unique()[0]

In [25]:
example_file

'./data\\2024-02-15_13h38m_Subject RCE3_1.txt'

In [26]:
example_one_session_recording_df = recording_and_metadata_df[recording_and_metadata_df["file_path"] == example_file]

- Everything looks normal enough

In [27]:
example_one_session_recording_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,85.25,64.0,399.0,0.0,60.01,86.12,1.0,7.0,54.0,13.0,32.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
1,86.29,144.0,399.0,0.0,140.01,86.40,1.0,27.0,55.0,13.0,33.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
2,86.63,234.0,399.0,0.0,230.01,86.65,1.0,57.0,56.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
3,86.72,314.0,399.0,0.0,310.01,86.74,1.0,17.0,58.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
4,86.83,389.0,399.0,0.0,385.01,86.86,1.0,32.0,59.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt


- `(S)CSpresentation` is the time that the tone starts playing. MED-PC starts inputting in random numbers that are in the 1000's. So we will remove these

In [28]:
example_one_session_recording_df[50:60]

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
50,206.39,NaN,NaN,0.0,12000.0,208.98,0.0,0.0,0.0,0.0,34.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
51,209.02,NaN,NaN,0.0,9000.0,209.18,0.0,0.0,0.0,0.0,36.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
52,209.20,NaN,NaN,0.0,8500.0,209.24,0.0,0.0,0.0,0.0,36.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
53,209.28,NaN,NaN,0.0,9000.0,209.33,0.0,0.0,0.0,0.0,36.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
54,209.48,NaN,NaN,0.0,9500.0,209.67,0.0,0.0,0.0,0.0,36.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
55,209.70,NaN,NaN,0.0,12000.0,210.53,0.0,0.0,0.0,0.0,37.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
56,211.36,NaN,NaN,0.0,8000.0,211.95,0.0,0.0,0.0,0.0,39.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
57,212.05,NaN,NaN,0.0,9500.0,212.15,0.0,0.0,0.0,0.0,39.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
58,218.09,NaN,NaN,0.0,8000.0,218.25,0.0,0.0,0.0,0.0,45.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
59,218.27,NaN,NaN,0.0,9000.0,219.58,0.0,0.0,0.0,0.0,45.0,56.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt


- `(S)CSpresentation` then starts adding blanks(NaN) so we will remove these too

In [29]:
example_one_session_recording_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
2503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
2504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
2505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1386.0,NaN,NaN,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
2506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
2507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt


- We will use a function that removes the `NaN` and the tone times that are divisible by 1000. This will get the tones time that were actually used for the recordings

In [30]:
example_one_session_valid_tones = processing.tone.get_valid_tones(example_one_session_recording_df["(S)CSpresentation"])

In [31]:
example_one_session_valid_tones

0       60.01
1      140.01
2      230.01
3      310.01
4      385.01
5      485.01
6      580.01
7      670.01
8      750.01
9      840.01
10     940.01
11    1030.01
12    1150.01
13    1240.01
14    1325.01
15    1415.01
16    1510.01
17    1630.01
18    1710.01
19    1805.01
20    1885.01
21    1975.01
22    2055.01
23    2130.01
24    2230.01
25    2325.01
26    2415.01
27    2495.01
28    2585.01
29    2685.01
30    2775.01
31    2895.01
32    2985.01
33    3070.01
34    3160.01
35    3255.01
36    3345.01
37    3425.01
38    3515.01
Name: (S)CSpresentation, dtype: float64

# Getting the First Port Entry After Each Tone

- With these tone playing times, we will get the first port entry that comes after the tone playing time
    - This was done by getting all the port entries that came after the tone. And then getting the port entry time that was earliest in time(aka smallest number)

In [32]:
processing.tone.get_first_port_entries_after_tone(tone_pd_series=example_one_session_valid_tones, port_entries_pd_series=example_one_session_recording_df["(P)Portentry"], port_exits_pd_series=example_one_session_recording_df["(N)Portexit"])

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone
0,60.01,85.25,86.12
1,140.01,140.80,141.06
2,230.01,231.48,231.51
3,310.01,314.61,314.83
4,385.01,386.41,386.51
5,485.01,486.95,487.63
6,580.01,585.10,585.20
7,670.01,673.43,685.94
8,750.01,756.97,757.07
9,840.01,849.99,856.50


- Getting the first port entry times for all the sessions

In [33]:
recording_and_metadata_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,85.25,64.0,399.0,0.0,60.01,86.12,1.0,7.0,54.0,13.0,32.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
1,86.29,144.0,399.0,0.0,140.01,86.40,1.0,27.0,55.0,13.0,33.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
2,86.63,234.0,399.0,0.0,230.01,86.65,1.0,57.0,56.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
3,86.72,314.0,399.0,0.0,310.01,86.74,1.0,17.0,58.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
4,86.83,389.0,399.0,0.0,385.01,86.86,1.0,32.0,59.0,13.0,34.0,54.0,13.0,20240215,RCE3_1,./data\2024-02-15_13h38m_Subject RCE3_1.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,20240305,RCE5_4,./data\2024-03-05_14h19m_Subject RCE5_4.txt
6004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,20240305,RCE5_4,./data\2024-03-05_14h19m_Subject RCE5_4.txt
6005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1775.0,NaN,NaN,20240305,RCE5_4,./data\2024-03-05_14h19m_Subject RCE5_4.txt
6006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,20240305,RCE5_4,./data\2024-03-05_14h19m_Subject RCE5_4.txt


In [34]:
concatted_first_porty_entry_dataframe = processing.tone.get_concatted_first_porty_entry_after_tone_dataframe(concatted_medpc_df=recording_and_metadata_df, date_column="date")

No valid tones for ./data\2024-02-26_15h19m_Subject RCE4_3.txt
No valid tones for ./data\2024-02-26_15h19m_Subject RCE4_4.txt


In [35]:
concatted_first_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject
0,60.01,85.25,86.12,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1
1,140.01,140.80,141.06,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1
3,310.01,314.61,314.83,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1
4,385.01,386.41,386.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1


In [36]:
concatted_first_porty_entry_dataframe.tail()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject
3781,3160.01,3160.07,3160.60,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4
3782,3255.01,3255.02,3255.53,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4
3783,3345.01,3345.17,3345.32,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4
3784,3425.01,3431.19,3431.22,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4
3785,3515.01,3518.45,3518.96,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4


# Getting the Last Port Entry Before the Tone

- We will do the same, but get the port entries that happened before the tone

In [37]:
port_entries_pd_series = example_one_session_recording_df["(P)Portentry"]
port_exit_pd_series = example_one_session_recording_df["(N)Portexit"]

- Example run for one session

In [38]:
example_last_port_entries_before_tone = processing.tone.get_last_port_entries_before_tone(tone_pd_series=example_one_session_valid_tones, port_entries_pd_series=port_entries_pd_series, port_exits_pd_series=port_exit_pd_series)

In [39]:
example_last_port_entries_before_tone

,current_tone_time,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone
0,60.01,NaN,NaN
1,140.01,138.46,138.52
2,230.01,228.85,230.06
3,310.01,308.02,308.26
4,385.01,381.31,381.39
5,485.01,477.47,477.79
6,580.01,578.52,578.63
7,670.01,659.87,660.32
8,750.01,748.79,748.83
9,840.01,830.40,831.33


- Getting all the port entries that happen before the tone for all sessions

In [40]:
concatted_last_porty_entry_dataframe = processing.tone.get_concatted_last_porty_entry_before_tone_dataframe(concatted_medpc_df=recording_and_metadata_df, date_column="date")

No valid tones for ./data\2024-02-26_15h19m_Subject RCE4_3.txt
No valid tones for ./data\2024-02-26_15h19m_Subject RCE4_4.txt


In [41]:
concatted_last_porty_entry_dataframe.head()

,current_tone_time,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,file_path,date,subject
0,60.01,NaN,NaN,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1
1,140.01,138.46,138.52,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1
2,230.01,228.85,230.06,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1
3,310.01,308.02,308.26,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1
4,385.01,381.31,381.39,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1


## Merging the dataframes for the first port entry after the tone and the last tone before the tone 

In [42]:
# Combining it based on the tone times and the file path
concatted_first_and_last_porty_entry_dataframe = pd.merge(concatted_first_porty_entry_dataframe, concatted_last_porty_entry_dataframe,  how='left', left_on=['current_tone_time','file_path'], right_on = ['current_tone_time','file_path'], suffixes=('', '_y'))
# Dropping all columns that are duplicates
concatted_first_and_last_porty_entry_dataframe = concatted_first_and_last_porty_entry_dataframe.drop(concatted_first_and_last_porty_entry_dataframe.filter(regex='_y$').columns.tolist(), axis=1)

- Adding the cage and strain information

In [43]:
concatted_first_and_last_porty_entry_dataframe["subject"] = concatted_first_and_last_porty_entry_dataframe["subject"].astype(str)
recording_and_metadata_df["subject"] = recording_and_metadata_df["subject"].astype(str)
# metadata_df["Subject"] = metadata_df["Subject"].astype(str)


In [44]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone
0,60.01,85.25,86.12,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,NaN,NaN
1,140.01,140.80,141.06,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,138.46,138.52
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,228.85,230.06
3,310.01,314.61,314.83,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,308.02,308.26
4,385.01,386.41,386.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,381.31,381.39


In [45]:
concatted_first_and_last_porty_entry_dataframe.shape

(3786, 8)

- Getting all the rows that have port entry to port exit time that overlaps with the tone playing
    - This would be the port entries that came before the tone, but had a port exit after

In [46]:
port_entries_that_overlap = concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["port_exit_after_last_port_entry_before_tone"] >= concatted_first_and_last_porty_entry_dataframe["current_tone_time"]]

In [47]:
port_entries_that_overlap.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,228.85,230.06
12,1150.01,1163.61,1168.25,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,1149.64,1150.08
17,1630.01,1636.40,1642.79,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,1629.81,1630.33
23,2130.01,2137.90,2137.93,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,2129.74,2130.92
42,310.01,310.59,310.67,./data\2024-02-15_13h38m_Subject RCE3_3.txt,20240215,RCE3_3,309.97,310.33


In [48]:
port_entries_that_overlap.shape

(843, 8)

# Getting the Latency for Each Session of a Tone Playing

- To calculate latency, we would just subtract the time of the first port entry to that of the tone playing

In [49]:
concatted_first_and_last_porty_entry_dataframe["latency"] = concatted_first_porty_entry_dataframe["first_port_entry_after_tone"] - concatted_first_and_last_porty_entry_dataframe["current_tone_time"]

In [50]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency
0,60.01,85.25,86.12,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,NaN,NaN,25.24
1,140.01,140.80,141.06,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,138.46,138.52,0.79
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,228.85,230.06,1.47
3,310.01,314.61,314.83,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,308.02,308.26,4.60
4,385.01,386.41,386.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,381.31,381.39,1.40


- For all the latencies that are are greater than 30 seconds, we changed the latency to 30 seconds

In [51]:
concatted_first_and_last_porty_entry_dataframe["latency_adjusted_greater_than_30"] = concatted_first_and_last_porty_entry_dataframe["latency"].apply(lambda x: 30 if x >= 30 else x)

In [52]:
concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["latency"] >= 30].head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30
46,670.01,746.94,747.12,./data\2024-02-15_13h38m_Subject RCE3_3.txt,20240215,RCE3_3,661.66,662.00,76.93,30.0
56,1630.01,1686.47,1686.49,./data\2024-02-15_13h38m_Subject RCE3_3.txt,20240215,RCE3_3,1605.51,1605.59,56.46,30.0
117,60.01,94.52,94.59,./data\2024-02-15_13h38m_Subject RCE4_2.txt,20240215,RCE4_2,28.42,28.51,34.51,30.0
123,580.01,695.32,695.37,./data\2024-02-15_13h38m_Subject RCE4_2.txt,20240215,RCE4_2,508.46,508.54,115.31,30.0
127,940.01,975.68,975.71,./data\2024-02-15_13h38m_Subject RCE4_2.txt,20240215,RCE4_2,920.99,921.22,35.67,30.0


# Seeing which port entries and port exit durations overlap with the tone

- Making a column that states whether or not the last port entry before the tone has a port exit after the tone. This would mean that they are overlapping.

In [53]:
concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"] = concatted_first_and_last_porty_entry_dataframe.apply(lambda row: True if row["port_exit_after_last_port_entry_before_tone"] >= row["current_tone_time"] else False, axis=1)


In [54]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone
0,60.01,85.25,86.12,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,NaN,NaN,25.24,25.24,False
1,140.01,140.80,141.06,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,138.46,138.52,0.79,0.79,False
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,228.85,230.06,1.47,1.47,True
3,310.01,314.61,314.83,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,308.02,308.26,4.60,4.60,False
4,385.01,386.41,386.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,381.31,381.39,1.40,1.40,False


- Comparing the latencies of the first port entry after the tone between these two groups:
    - 1. Those with the last port entry before the tone that has a port exit after the tone("overlapping" group)
    - 2. Those with the last port entry before the tone that has a port exit before the tone("nonoverlapping" group)

In [55]:
overlapping_df = concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"]]

In [56]:
non_overlapping_df = concatted_first_and_last_porty_entry_dataframe[~concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"]]

- Number of rows of overlapping and nonoverlapping last port entries before the tone

In [57]:
overlapping_df.shape

(843, 11)

In [58]:
non_overlapping_df.shape

(2943, 11)

- Latency between the tone playing and the first port entry after the tone

In [59]:
overlapping_df["latency"].mean()

np.float64(1.8270937129299172)

In [60]:
non_overlapping_df["latency"].mean()

np.float64(6.27474633980243)

- Latency between the tone playing and the first port entry after the tone. This had been adjusted so that all latencies that are greater than 30 seconds were adjusted to be just 30 seconds.

In [61]:
overlapping_df["latency_adjusted_greater_than_30"].mean()

np.float64(1.7942941874257663)

In [62]:
non_overlapping_df["latency_adjusted_greater_than_30"].mean()

np.float64(4.686601974804135)

# Adjusting the Latencies if the Port Entries/Exit Overlap With the Tone Times

- For all the rows with overlapping last port entries before the tone, we will adjust the latency of the tone to the first port entry after the tone to 0. (Because the first port entry after the tone would be considered to be the same as the last port entry before the tone)

In [63]:
concatted_first_and_last_porty_entry_dataframe["latency_adjusted_overlap"] = concatted_first_and_last_porty_entry_dataframe.apply(lambda row: 0 if row["overlapping_port_entry_with_tone"] else row["latency_adjusted_greater_than_30"], axis=1)

In [64]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap
0,60.01,85.25,86.12,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,NaN,NaN,25.24,25.24,False,25.24
1,140.01,140.80,141.06,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,138.46,138.52,0.79,0.79,False,0.79
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,228.85,230.06,1.47,1.47,True,0.00
3,310.01,314.61,314.83,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,308.02,308.26,4.60,4.60,False,4.60
4,385.01,386.41,386.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,381.31,381.39,1.40,1.40,False,1.40


- All the rows with overlapping last port entries before the tone

In [65]:
concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"]].head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,228.85,230.06,1.47,1.47,True,0.0
12,1150.01,1163.61,1168.25,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,1149.64,1150.08,13.60,13.60,True,0.0
17,1630.01,1636.40,1642.79,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,1629.81,1630.33,6.39,6.39,True,0.0
23,2130.01,2137.90,2137.93,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,2129.74,2130.92,7.89,7.89,True,0.0
42,310.01,310.59,310.67,./data\2024-02-15_13h38m_Subject RCE3_3.txt,20240215,RCE3_3,309.97,310.33,0.58,0.58,True,0.0


## Labeling if a row's latency is less than 10 seconds or not

- We will be getting the ratio of latencies(from the time that the tone played, to the first port entry after the tone) that are less than 10 seconds for each session. So we will label all the adjusted latencies that are less than 10 seconds as `True`, and the those with latencies that are greater than 10 seconds as `False` 

In [66]:
concatted_first_and_last_porty_entry_dataframe["adjusted_30_second_latency_less_than_10_seconds_ratio"] = concatted_first_and_last_porty_entry_dataframe["latency_adjusted_greater_than_30"].apply(lambda x: True if x <= 10 else False)

In [67]:
concatted_first_and_last_porty_entry_dataframe["overlap_adjusted_latency_less_than_10_seconds"] = concatted_first_and_last_porty_entry_dataframe["latency_adjusted_overlap"].apply(lambda x: True if x <= 10 else False)

In [68]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap,adjusted_30_second_latency_less_than_10_seconds_ratio,overlap_adjusted_latency_less_than_10_seconds
0,60.01,85.25,86.12,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,NaN,NaN,25.24,25.24,False,25.24,False,False
1,140.01,140.80,141.06,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,138.46,138.52,0.79,0.79,False,0.79,True,True
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,228.85,230.06,1.47,1.47,True,0.00,True,True
3,310.01,314.61,314.83,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,308.02,308.26,4.60,4.60,False,4.60,True,True
4,385.01,386.41,386.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,381.31,381.39,1.40,1.40,False,1.40,True,True


# Making Dataframes that have Each Subject as the Row and Each Date as the Column

- Pivot plots allow us to make columns out of each unique entry in a selected column. (For this, it will be the date column that we will turn into columns) The index for the rows will be each unique entry in another column. (For this, it will be the subject's ID) And the values will be the aggregate of the values that have both the values in the previous two selected columns. (For this it will be the latency columns and we will take the mean)
    - https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

- Making a folder for all the pivotplots

# NOTE: If you decide to change the cage numbers or the dates used from the original ones from the imported dataframes, you will need to change the subsequent path to the cage number and dates you are using

In [69]:
FILE_PREFIX = "rce_training"

In [70]:
first_port_entry_to_tone_latency_output_directory = "./proc/subject_rows_with_date_columns_dataframes/first_port_entry_to_tone_latency/{}_{}_{}".format(FILE_PREFIX, earliest_date, latest_date)

In [71]:
first_port_entry_to_tone_latency_output_directory

'./proc/subject_rows_with_date_columns_dataframes/first_port_entry_to_tone_latency/rce_training_20240215_20240305'

In [72]:
os.makedirs(first_port_entry_to_tone_latency_output_directory, exist_ok=True)

In [73]:
latencies_less_than_10_seconds_ratio_output_directory = "./proc/subject_rows_with_date_columns_dataframes/latencies_less_than_10_seconds_ratio/{}_{}_{}".format(FILE_PREFIX, earliest_date, latest_date)

In [74]:
latencies_less_than_10_seconds_ratio_output_directory

'./proc/subject_rows_with_date_columns_dataframes/latencies_less_than_10_seconds_ratio/rce_training_20240215_20240305'

In [75]:
os.makedirs(latencies_less_than_10_seconds_ratio_output_directory, exist_ok=True)

# Calculating the dates

In [89]:
subject_to_all_dates = concatted_first_and_last_porty_entry_dataframe.groupby("subject")["date"].apply(set).reset_index(name='all_dates')

In [90]:
subject_to_all_dates["all_dates"] = subject_to_date["all_dates"].apply(lambda x: sorted(list(x)))

In [92]:
subject_to_all_dates["date_to_index"] = subject_to_date["all_dates"].apply(lambda x: {date: index for index, date in enumerate(x)})

In [94]:
subject_to_all_dates

,subject,all_dates,date_to_index
0,RCE3_1,"[20240215, 20240216, 20240217, 20240218, 20240...","{20240215: 0, 20240216: 1, 20240217: 2, 202402..."
1,RCE3_3,"[20240215, 20240216, 20240217, 20240218, 20240...","{20240215: 0, 20240216: 1, 20240217: 2, 202402..."
2,RCE3_4,"[20240215, 20240216, 20240217, 20240218, 20240...","{20240215: 0, 20240216: 1, 20240217: 2, 202402..."
3,RCE4_2,"[20240215, 20240216, 20240217, 20240218, 20240...","{20240215: 0, 20240216: 1, 20240217: 2, 202402..."
4,RCE4_3,"[20240215, 20240216, 20240217, 20240218, 20240...","{20240215: 0, 20240216: 1, 20240217: 2, 202402..."
5,RCE4_4,"[20240215, 20240216, 20240217, 20240218, 20240...","{20240215: 0, 20240216: 1, 20240217: 2, 202402..."
6,RCE5_2,"[20240223, 20240224, 20240225, 20240226, 20240...","{20240223: 0, 20240224: 1, 20240225: 2, 202402..."
7,RCE5_3,"[20240223, 20240224, 20240225, 20240226, 20240...","{20240223: 0, 20240224: 1, 20240225: 2, 202402..."
8,RCE5_4,"[20240223, 20240224, 20240225, 20240226, 20240...","{20240223: 0, 20240224: 1, 20240225: 2, 202402..."


In [96]:
subject_to_date_index = pd.Series(subject_to_all_dates["date_to_index"].values,index=subject_to_all_dates["subject"]).to_dict()


In [97]:
subject_to_date_index

{'RCE3_1': {20240215: 0,
  20240216: 1,
  20240217: 2,
  20240218: 3,
  20240219: 4,
  20240220: 5,
  20240221: 6,
  20240222: 7,
  20240223: 8,
  20240225: 9,
  20240226: 10},
 'RCE3_3': {20240215: 0,
  20240216: 1,
  20240217: 2,
  20240218: 3,
  20240219: 4,
  20240220: 5,
  20240221: 6,
  20240222: 7,
  20240223: 8,
  20240225: 9,
  20240226: 10},
 'RCE3_4': {20240215: 0,
  20240216: 1,
  20240217: 2,
  20240218: 3,
  20240219: 4,
  20240220: 5,
  20240221: 6,
  20240222: 7,
  20240223: 8,
  20240225: 9,
  20240226: 10},
 'RCE4_2': {20240215: 0,
  20240216: 1,
  20240217: 2,
  20240218: 3,
  20240219: 4,
  20240220: 5,
  20240221: 6,
  20240222: 7,
  20240223: 8,
  20240225: 9,
  20240226: 10},
 'RCE4_3': {20240215: 0,
  20240216: 1,
  20240217: 2,
  20240218: 3,
  20240219: 4,
  20240220: 5,
  20240221: 6,
  20240222: 7,
  20240223: 8,
  20240225: 9,
  20240226: 10},
 'RCE4_4': {20240215: 0,
  20240216: 1,
  20240217: 2,
  20240218: 3,
  20240219: 4,
  20240220: 5,
  20240221: 6,


In [99]:
concatted_first_and_last_porty_entry_dataframe["date_index"] = concatted_first_and_last_porty_entry_dataframe.apply(lambda row: subject_to_date_index[row["subject"]][row["date"]], axis=1)

In [100]:
concatted_first_and_last_porty_entry_dataframe

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap,adjusted_30_second_latency_less_than_10_seconds_ratio,overlap_adjusted_latency_less_than_10_seconds,date_index
0,60.01,85.25,86.12,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,NaN,NaN,25.24,25.24,False,25.24,False,False,0
1,140.01,140.80,141.06,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,138.46,138.52,0.79,0.79,False,0.79,True,True,0
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,228.85,230.06,1.47,1.47,True,0.00,True,True,0
3,310.01,314.61,314.83,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,308.02,308.26,4.60,4.60,False,4.60,True,True,0
4,385.01,386.41,386.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,381.31,381.39,1.40,1.40,False,1.40,True,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781,3160.01,3160.07,3160.60,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4,3159.79,3159.87,0.06,0.06,False,0.06,True,True,10
3782,3255.01,3255.02,3255.53,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4,3254.39,3255.00,0.01,0.01,False,0.01,True,True,10
3783,3345.01,3345.17,3345.32,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4,3344.58,3344.83,0.16,0.16,False,0.16,True,True,10
3784,3425.01,3431.19,3431.22,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4,3419.82,3420.74,6.18,6.18,False,6.18,True,True,10


## Making a pivot plot with the original latency

In [101]:
all_latency_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency', \
                                        index=['subject'], columns=['date_index'], aggfunc=np.mean)


C:\Users\Ryo Iwata\AppData\Local\Temp\ipykernel_29320\3058721259.py:1: FutureWarning: The provided callable <function mean at 0x00000211BC2A5700> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  all_latency_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency', \


In [103]:
all_latency_pivot_plot

date_index,0,1,2,3,4,5,6,7,8,9,10
subject,,,,,,,,,,,
RCE3_1,6.308205,7.229231,2.488974,2.363077,2.269487,1.924103,1.436923,2.315385,1.308718,0.765641,2.414872
RCE3_3,10.073077,9.109231,3.887436,4.641795,2.367692,2.549231,2.476923,1.536410,1.481026,1.187692,1.481538
RCE3_4,8.423077,8.789744,9.136410,2.934872,2.014103,2.073846,1.509744,2.003077,4.177949,1.063077,0.757436
RCE4_2,13.115385,5.019744,6.288974,5.926923,2.101026,2.268205,2.317949,2.088974,2.611795,1.790513,1.517949
RCE4_3,9.274103,9.381026,3.966667,3.486667,1.456923,1.601026,1.209231,7.802821,1.160513,0.816410,0.866923
RCE4_4,9.335128,6.530000,5.838718,3.721026,2.681026,1.716667,33.152368,2.938974,2.627692,3.424615,1.808462
RCE5_2,68.616923,6.199231,1.955897,5.964615,3.566667,2.710256,2.114103,1.549231,1.984359,6.147692,NaN
RCE5_3,7.161795,3.671026,5.859744,2.024615,2.306923,1.458718,1.854615,1.181282,2.084615,2.115128,NaN
RCE5_4,75.829189,8.234359,7.708205,5.440256,2.505641,1.272308,1.290513,1.964872,2.049744,3.596410,2.005385


- Getting the cage numbers for each subject and then making a dictionary out of it
    - This will be used to add the cage information to the pivot plots

In [104]:
# Converting the cage numbers from floats to integers
# When the metadata is imported from the csv files, the cage number is imported as a float
# metadata_df["cage"] = metadata_df["cage"].astype(int)

- Adding the cage information with the dictionary

In [105]:
all_latency_pivot_plot.head()

date_index,0,1,2,3,4,5,6,7,8,9,10
subject,,,,,,,,,,,
RCE3_1,6.308205,7.229231,2.488974,2.363077,2.269487,1.924103,1.436923,2.315385,1.308718,0.765641,2.414872
RCE3_3,10.073077,9.109231,3.887436,4.641795,2.367692,2.549231,2.476923,1.536410,1.481026,1.187692,1.481538
RCE3_4,8.423077,8.789744,9.136410,2.934872,2.014103,2.073846,1.509744,2.003077,4.177949,1.063077,0.757436
RCE4_2,13.115385,5.019744,6.288974,5.926923,2.101026,2.268205,2.317949,2.088974,2.611795,1.790513,1.517949
RCE4_3,9.274103,9.381026,3.966667,3.486667,1.456923,1.601026,1.209231,7.802821,1.160513,0.816410,0.866923


In [106]:
file_path = "original_latency_{}_{}_{}.csv".format(FILE_PREFIX, earliest_date, latest_date)
output_path = os.path.join(first_port_entry_to_tone_latency_output_directory, file_path)
try:
    all_latency_pivot_plot.to_csv(output_path)
except:
    output_path = os.path.abspath(output_path)
    # Changing path if using on Windows because it raises an error if the path is longer than 260 characters
    # But adding this suffix bypasses this
    output_path = u"\\\\?\\" + output_path
    all_latency_pivot_plot.to_csv(output_path)

## Making a pivot plot with the latency that were greater than 30 seconds being adjusted 

In [107]:
latency_greater_than_30_seconds_adjusted_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency_adjusted_greater_than_30', index=['subject'],

                    columns=['date_index'], aggfunc=np.mean)


C:\Users\Ryo Iwata\AppData\Local\Temp\ipykernel_29320\361944485.py:1: FutureWarning: The provided callable <function mean at 0x00000211BC2A5700> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  latency_greater_than_30_seconds_adjusted_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency_adjusted_greater_than_30', index=['subject'],


In [108]:
latency_greater_than_30_seconds_adjusted_pivot_plot.head()

date_index,0,1,2,3,4,5,6,7,8,9,10
subject,,,,,,,,,,,
RCE3_1,6.308205,7.229231,2.488974,2.363077,2.269487,1.924103,1.436923,2.315385,1.308718,0.765641,2.414872
RCE3_3,8.191282,9.070769,3.887436,4.641795,2.367692,2.549231,2.476923,1.536410,1.481026,1.187692,1.481538
RCE3_4,8.423077,8.682564,8.549487,2.934872,2.014103,2.073846,1.509744,2.003077,4.177949,1.063077,0.757436
RCE4_2,10.666923,5.019744,6.288974,5.809231,2.101026,2.268205,2.317949,2.088974,2.611795,1.790513,1.517949
RCE4_3,8.819487,8.994872,3.966667,3.486667,1.456923,1.601026,1.209231,7.654872,1.160513,0.816410,0.866923


In [109]:
file_path = "subject_to_date_latency_greater_than_30_seconds_adjusted_{}_date_{}_{}.csv".format(FILE_PREFIX, earliest_date, latest_date)
output_path = os.path.join(first_port_entry_to_tone_latency_output_directory, file_path)
try:
    latency_greater_than_30_seconds_adjusted_pivot_plot.to_csv(output_path)
except:
    output_path = os.path.abspath(output_path)
    # Changing path if using on Windows because it raises an error if the path is longer than 260 characters
    # But adding this suffix bypasses this
    output_path = u"\\\\?\\" + output_path
    latency_greater_than_30_seconds_adjusted_pivot_plot.to_csv(output_path)

## Making a pivot plot with adjusted latencies that overlapped with a tone playing

In [110]:
latency_that_overlaps_with_tone_adjusted_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency_adjusted_overlap', index=['subject'],

                    columns=['date_index'], aggfunc=np.mean)


C:\Users\Ryo Iwata\AppData\Local\Temp\ipykernel_29320\1364428807.py:1: FutureWarning: The provided callable <function mean at 0x00000211BC2A5700> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  latency_that_overlaps_with_tone_adjusted_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency_adjusted_overlap', index=['subject'],


In [111]:
latency_that_overlaps_with_tone_adjusted_pivot_plot.head()

date_index,0,1,2,3,4,5,6,7,8,9,10
subject,,,,,,,,,,,
RCE3_1,5.555641,7.014615,2.148974,1.711026,1.874103,1.220000,0.877692,0.757436,0.323590,0.346923,1.169487
RCE3_3,8.143590,8.717436,3.873077,4.331026,2.042308,2.195128,2.166667,1.424615,1.398974,0.980513,1.110000
RCE3_4,8.423077,8.673077,8.545385,2.781795,1.755385,1.945385,1.315128,1.659231,4.160769,0.484615,0.487436
RCE4_2,10.544615,4.960000,6.127692,5.467692,1.917692,2.168718,1.974615,1.973590,2.067949,1.618974,1.412821
RCE4_3,8.812821,8.775897,3.850513,2.975897,1.137436,1.197179,1.036667,6.264872,0.931795,0.267949,0.355897


In [112]:
file_path = "subject_to_date_latency_that_overlaps_with_tone_adjusted_{}_date_{}_{}.csv".format(FILE_PREFIX, earliest_date, latest_date)
output_path = os.path.join(first_port_entry_to_tone_latency_output_directory, file_path)
try:
    latency_that_overlaps_with_tone_adjusted_pivot_plot.to_csv(output_path)
except:
    output_path = os.path.abspath(output_path)
    # Changing path if using on Windows because it raises an error if the path is longer than 260 characters
    # But adding this suffix bypasses this
    output_path = u"\\\\?\\" + output_path
    latency_that_overlaps_with_tone_adjusted_pivot_plot.to_csv(output_path)

## Making a pivot plot with the ratio of original latencies that are less than 10 seconds

In [113]:
original_less_than_10_latency_df = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='adjusted_30_second_latency_less_than_10_seconds_ratio', index=['subject'],

                    columns=['date_index'], aggfunc=np.mean)


C:\Users\Ryo Iwata\AppData\Local\Temp\ipykernel_29320\2635638577.py:1: FutureWarning: The provided callable <function mean at 0x00000211BC2A5700> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  original_less_than_10_latency_df = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='adjusted_30_second_latency_less_than_10_seconds_ratio', index=['subject'],


In [114]:
original_less_than_10_latency_df

date_index,0,1,2,3,4,5,6,7,8,9,10
subject,,,,,,,,,,,
RCE3_1,0.820513,0.846154,0.974359,1.000000,1.000000,1.000000,1.000000,0.974359,1.000000,1.000000,0.974359
RCE3_3,0.769231,0.564103,0.974359,0.897436,0.948718,1.000000,0.974359,1.000000,1.000000,1.000000,0.974359
RCE3_4,0.615385,0.564103,0.692308,0.974359,0.974359,0.974359,1.000000,1.000000,0.871795,0.974359,1.000000
RCE4_2,0.538462,0.769231,0.850000,0.846154,1.000000,1.000000,1.000000,1.000000,0.974359,1.000000,0.948718
RCE4_3,0.666667,0.743590,0.975000,0.948718,1.000000,0.948718,1.000000,0.769231,1.000000,1.000000,1.000000
RCE4_4,0.615385,0.897436,0.875000,0.974359,1.000000,1.000000,0.307692,1.000000,0.923077,0.974359,1.000000
RCE5_2,0.307692,0.794872,1.000000,0.769231,0.948718,1.000000,1.000000,1.000000,0.948718,0.820513,NaN
RCE5_3,0.769231,0.923077,0.897436,1.000000,1.000000,1.000000,0.974359,1.000000,0.974359,1.000000,NaN
RCE5_4,0.179487,0.589744,0.769231,0.820513,1.000000,1.000000,1.000000,0.974359,1.000000,0.974359,0.974359


In [115]:
file_path = "subject_to_date_less_than_10_seconds_original_latencies_ratios_{}_date_{}_{}.csv".format(FILE_PREFIX, earliest_date, latest_date)
output_path = os.path.join(first_port_entry_to_tone_latency_output_directory, file_path)
try:
    original_less_than_10_latency_df.to_csv(output_path)
except:
    output_path = os.path.abspath(output_path)
    # Changing path if using on Windows because it raises an error if the path is longer than 260 characters
    # But adding this suffix bypasses this
    output_path = u"\\\\?\\" + output_path
    original_less_than_10_latency_df.to_csv(output_path)

## Making a pivot plot with the ratio of overlap adjusted latencies that are less than 10 seconds

In [116]:
overlap_adjusted_less_than_10_latency_df = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='overlap_adjusted_latency_less_than_10_seconds', index=['subject'],

                    columns=['date_index'], aggfunc=np.mean)


C:\Users\Ryo Iwata\AppData\Local\Temp\ipykernel_29320\361287888.py:1: FutureWarning: The provided callable <function mean at 0x00000211BC2A5700> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  overlap_adjusted_less_than_10_latency_df = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='overlap_adjusted_latency_less_than_10_seconds', index=['subject'],


In [117]:
overlap_adjusted_less_than_10_latency_df

date_index,0,1,2,3,4,5,6,7,8,9,10
subject,,,,,,,,,,,
RCE3_1,0.846154,0.846154,0.974359,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.974359
RCE3_3,0.769231,0.589744,0.974359,0.897436,0.948718,1.000000,0.974359,1.000000,1.000000,1.000000,0.974359
RCE3_4,0.615385,0.564103,0.692308,0.974359,0.974359,0.974359,1.000000,1.000000,0.871795,1.000000,1.000000
RCE4_2,0.538462,0.769231,0.850000,0.846154,1.000000,1.000000,1.000000,1.000000,0.974359,1.000000,0.948718
RCE4_3,0.666667,0.743590,0.975000,0.974359,1.000000,0.948718,1.000000,0.820513,1.000000,1.000000,1.000000
RCE4_4,0.615385,0.897436,0.875000,0.974359,1.000000,1.000000,0.307692,1.000000,1.000000,1.000000,1.000000
RCE5_2,0.333333,0.794872,1.000000,0.974359,0.948718,1.000000,1.000000,1.000000,0.948718,0.948718,NaN
RCE5_3,0.794872,0.923077,0.897436,1.000000,1.000000,1.000000,0.974359,1.000000,0.974359,1.000000,NaN
RCE5_4,0.179487,0.615385,0.769231,0.820513,1.000000,1.000000,1.000000,0.974359,1.000000,0.974359,0.974359


In [118]:
latencies_less_than_10_seconds_ratio_output_directory

'./proc/subject_rows_with_date_columns_dataframes/latencies_less_than_10_seconds_ratio/rce_training_20240215_20240305'

In [119]:
overlap_adjusted_less_than_10_latency_df.to_csv("/".join(["./proc/subject_rows_with_date_columns_dataframes", "subject_to_date_adjusted_latencies_ratios_{}_date_{}_{}.csv".format(FILE_PREFIX, earliest_date, latest_date)]))
overlap_adjusted_less_than_10_latency_df.to_excel("/".join(["./proc/subject_rows_with_date_columns_dataframes", "subject_to_date_adjusted_latencies_ratios_{}_date_{}_{}.xlsx".format(FILE_PREFIX, earliest_date, latest_date)]))

# Plotting the latencies

## Getting the averages for all the trials in one subject and one day

- We will plot a line for each subject. With the X-axis being the days since the first session. And the Y-axis is the latency values averaged across all the trials for one recording session
- Grouping all the rows with the same subject and date(aka, all the trials in one session). And then getting the mean for each value

In [120]:
float_columns = concatted_first_and_last_porty_entry_dataframe.select_dtypes(include=[np.float64]).columns

In [121]:
float_columns

Index(['current_tone_time', 'first_port_entry_after_tone',
       'port_exit_after_first_port_entry_after_tone',
       'last_port_entry_before_tone',
       'port_exit_after_last_port_entry_before_tone', 'latency',
       'latency_adjusted_greater_than_30', 'latency_adjusted_overlap'],
      dtype='object')

In [125]:
grouped_averaged_first_porty_entry_dataframe = concatted_first_and_last_porty_entry_dataframe[list(float_columns) + ["date_index", "subject"]].groupby(by=["date_index", "subject"]).mean()

In [126]:
concatted_first_and_last_porty_entry_dataframe

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap,adjusted_30_second_latency_less_than_10_seconds_ratio,overlap_adjusted_latency_less_than_10_seconds,date_index
0,60.01,85.25,86.12,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,NaN,NaN,25.24,25.24,False,25.24,False,False,0
1,140.01,140.80,141.06,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,138.46,138.52,0.79,0.79,False,0.79,True,True,0
2,230.01,231.48,231.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,228.85,230.06,1.47,1.47,True,0.00,True,True,0
3,310.01,314.61,314.83,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,308.02,308.26,4.60,4.60,False,4.60,True,True,0
4,385.01,386.41,386.51,./data\2024-02-15_13h38m_Subject RCE3_1.txt,20240215,RCE3_1,381.31,381.39,1.40,1.40,False,1.40,True,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781,3160.01,3160.07,3160.60,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4,3159.79,3159.87,0.06,0.06,False,0.06,True,True,10
3782,3255.01,3255.02,3255.53,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4,3254.39,3255.00,0.01,0.01,False,0.01,True,True,10
3783,3345.01,3345.17,3345.32,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4,3344.58,3344.83,0.16,0.16,False,0.16,True,True,10
3784,3425.01,3431.19,3431.22,./data\2024-03-05_14h19m_Subject RCE5_4.txt,20240305,RCE5_4,3419.82,3420.74,6.18,6.18,False,6.18,True,True,10


In [127]:
grouped_averaged_first_porty_entry_dataframe.head()

current_tone_time  first_port_entry_after_tone  \
date_index subject                                                   
0          RCE3_1          1779.75359                  1786.061795   
           RCE3_3          1779.75359                  1789.826667   
           RCE3_4          1779.75359                  1788.176667   
           RCE4_2          1779.75359                  1792.868974   
           RCE4_3          1779.75359                  1789.027692   

                    port_exit_after_first_port_entry_after_tone  \
date_index subject                                                
0          RCE3_1                                   1787.705385   
           RCE3_3                                   1790.110000   
           RCE3_4                                   1789.507436   
           RCE4_2                                   1792.936154   
           RCE4_3                                   1789.101282   

                    last_port_entry_before_tone  \
date_index subject                                
0          RCE3_1                   1820.065789   
           RCE3_3                   1767.428205   
           RCE3_4                   1767.007436   
           RCE4_2                   1755.843846   
           RCE4_3                   1813.020789   

                    port_exit_after_last_port_entry_before_tone    latency  \
date_index subject                                                           
0          RCE3_1                                   1820.667632   6.308205   
           RCE3_3                                   1767.750769  10.073077   
           RCE3_4                                   1767.524615   8.423077   
           RCE4_2                                   1756.075641  13.115385   
           RCE4_3                                   1813.209211   9.274103   

                    latency_adjusted_greater_than_30  latency_adjusted_overlap  
date_index subject                                                              
0          RCE3_1                           6.308205                  5.555641  
           RCE3_3                           8.191282                  8.143590  
           RCE3_4                           8.423077                  8.423077  
           RCE4_2                          10.666923                 10.544615  
           RCE4_3                           8.819487                  8.812821

## Reformatting the Dataframe for plotting

- Resetting the index so that we can use the data as a value

In [128]:
grouped_averaged_first_porty_entry_dataframe = grouped_averaged_first_porty_entry_dataframe.reset_index()

In [129]:
grouped_averaged_first_porty_entry_dataframe.head()

,date_index,subject,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,latency_adjusted_overlap
0,0,RCE3_1,1779.75359,1786.061795,1787.705385,1820.065789,1820.667632,6.308205,6.308205,5.555641
1,0,RCE3_3,1779.75359,1789.826667,1790.110000,1767.428205,1767.750769,10.073077,8.191282,8.143590
2,0,RCE3_4,1779.75359,1788.176667,1789.507436,1767.007436,1767.524615,8.423077,8.423077,8.423077
3,0,RCE4_2,1779.75359,1792.868974,1792.936154,1755.843846,1756.075641,13.115385,10.666923,10.544615
4,0,RCE4_3,1779.75359,1789.027692,1789.101282,1813.020789,1813.209211,9.274103,8.819487,8.812821


- Turning the date column which is a string, into an integer
    - This will allow us to use the date as the X-axis for plotting

- Adding the cage information
    - We will make one plot for each cage

In [130]:
grouped_averaged_first_porty_entry_dataframe["subject"].unique()

array(['RCE3_1', 'RCE3_3', 'RCE3_4', 'RCE4_2', 'RCE4_3', 'RCE4_4',
       'RCE5_2', 'RCE5_3', 'RCE5_4'], dtype=object)

In [131]:
grouped_averaged_first_porty_entry_dataframe.head()

,date_index,subject,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,latency_adjusted_overlap
0,0,RCE3_1,1779.75359,1786.061795,1787.705385,1820.065789,1820.667632,6.308205,6.308205,5.555641
1,0,RCE3_3,1779.75359,1789.826667,1790.110000,1767.428205,1767.750769,10.073077,8.191282,8.143590
2,0,RCE3_4,1779.75359,1788.176667,1789.507436,1767.007436,1767.524615,8.423077,8.423077,8.423077
3,0,RCE4_2,1779.75359,1792.868974,1792.936154,1755.843846,1756.075641,13.115385,10.666923,10.544615
4,0,RCE4_3,1779.75359,1789.027692,1789.101282,1813.020789,1813.209211,9.274103,8.819487,8.812821


## Plotting the latencies that were adjusted if they greater than 30 seconds

In [132]:
original_average_latency_output_directory = "./proc/plots/original_average_latency_plots/{}_date_{}_{}".format(FILE_PREFIX, earliest_date, latest_date)

In [133]:
original_average_latency_output_directory

'./proc/plots/original_average_latency_plots/rce_training_date_20240215_20240305'

In [134]:
os.makedirs(original_average_latency_output_directory, exist_ok=True)

In [135]:
plt.rcParams["figure.figsize"] = (13.5,7.5)

In [136]:
# Plotting for each cage
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (13.5,7.5)

# Getting all the rows with the current cage number
cage_df = grouped_averaged_first_porty_entry_dataframe

# Plotting a line for each subject
for subject in cage_df["subject"].unique():
    # Getting all the rows with the current subject
    subject_df = cage_df[cage_df["subject"] == subject]
    # Making the dates into days after the first session by subtracting all the dates by the first date
    ax.plot(subject_df["date_index"], subject_df["latency_adjusted_greater_than_30"], '-o', label=subject)

# Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
ax.set_ylim(0, 30)
# Labeling the X/Y Axis and the title
ax.set_xlabel("The Days After the First Session")
ax.set_ylabel("Adjusted Average Latency of First Entry to Tone Onset")
ax.set_title("Latency of Port Entry to Tone")
# To show the legend
ax.legend()
file_name = "original_average_latency_plot_{}_date_{}_{}.png".format(FILE_PREFIX, earliest_date, latest_date)
plt.savefig(os.path.join(original_average_latency_output_directory, file_name))

## Plotting the original ratio of latencies that are less than 10 seconds

In [137]:
less_than_10_second_original_latency_ratio_plots_output_directory = "./proc/plots/less_than_10_second_original_latency_ratio/{}_date_{}_{}".format(FILE_PREFIX, earliest_date, latest_date)

In [138]:
less_than_10_second_original_latency_ratio_plots_output_directory

'./proc/plots/less_than_10_second_original_latency_ratio/rce_training_date_20240215_20240305'

In [139]:
os.makedirs(less_than_10_second_original_latency_ratio_plots_output_directory, exist_ok=True)

In [142]:
subject_df

,date_index,subject,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,latency_adjusted_overlap
0,0,RCE3_1,1779.75359,1786.061795,1787.705385,1820.065789,1820.667632,6.308205,6.308205,5.555641
9,1,RCE3_1,1779.75359,1786.982821,1787.047692,1765.313077,1765.433077,7.229231,7.229231,7.014615
18,2,RCE3_1,1779.75359,1782.242564,1782.437179,1775.793077,1776.255641,2.488974,2.488974,2.148974
27,3,RCE3_1,1779.75359,1782.116667,1785.866410,1775.266154,1776.402308,2.363077,2.363077,1.711026
36,4,RCE3_1,1779.75359,1782.023077,1783.212051,1775.877179,1776.978718,2.269487,2.269487,1.874103
45,5,RCE3_1,1779.75359,1781.677692,1782.453333,1777.441026,1778.657436,1.924103,1.924103,1.220000
54,6,RCE3_1,1779.75359,1781.190513,1781.755897,1778.163590,1779.253077,1.436923,1.436923,0.877692
63,7,RCE3_1,1779.75359,1782.068974,1784.623590,1776.086923,1779.365897,2.315385,2.315385,0.757436
72,8,RCE3_1,1779.75359,1781.062308,1781.715128,1777.851795,1780.308462,1.308718,1.308718,0.323590
81,9,RCE3_1,1779.75359,1780.519231,1781.002821,1778.690256,1779.716410,0.765641,0.765641,0.346923


In [143]:
# # Plotting for each cage
# fig, ax = plt.subplots()
# plt.rcParams["figure.figsize"] = (13.5,7.5)

# # Getting all the rows with the current cage number
# cage_df = grouped_averaged_first_porty_entry_dataframe

# # Plotting a line for each subject
# for subject in cage_df["subject"].unique():
#     # Getting all the rows with the current subject
#     subject_df = cage_df[cage_df["subject"] == subject]
#     # Making the dates into days after the first session by subtracting all the dates by the first date
#     ax.plot(subject_df["date_index"], subject_df["adjusted_30_second_latency_less_than_10_seconds_ratio"], '-o', label=subject)
# # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
# ax.set_ylim(0, 1)
# # Labeling the X/Y Axis and the title
# ax.set_xlabel("The Days After the First Session")
# ax.set_ylabel("Proportion of Latencies")
# ax.set_title("Less Than 10 Seconds Latencies from Tone Onset")
# # To show the legend
# ax.legend()

# file_name = "less_than_10_second_original_latency_ratio_plot_{}_date_{}_{}.png".format(FILE_PREFIX, earliest_date, latest_date)
# plt.savefig(os.path.join(less_than_10_second_original_latency_ratio_plots_output_directory, file_name))


# Plotting the Adjusted Latencies for Entries that Overlap with the Tone

## Plotting the Latencies 

In [144]:
overlap_adjusted_average_latency_plots_output_directory = "./proc/plots/overlap_adjusted_average_latency_plots/{}_date_{}_{}".format(FILE_PREFIX, earliest_date, latest_date)

In [145]:
overlap_adjusted_average_latency_plots_output_directory

'./proc/plots/overlap_adjusted_average_latency_plots/rce_training_date_20240215_20240305'

In [146]:
os.makedirs(overlap_adjusted_average_latency_plots_output_directory, exist_ok=True)

In [150]:
# Plotting for each cage
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (13.5,7.5)

# Getting all the rows with the current cage number
cage_df = grouped_averaged_first_porty_entry_dataframe

# Plotting a line for each subject
for subject in cage_df["subject"].unique():
    # Getting all the rows with the current subject
    subject_df = cage_df[cage_df["subject"] == subject]
    # Making the dates into days after the first session by subtracting all the dates by the first date
    ax.plot(subject_df["date_index"], subject_df["latency_adjusted_overlap"], '-o', label=subject)

# Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
ax.set_ylim(0, 30)
# Labeling the X/Y Axis and the title
ax.set_xlabel("The Days After the First Session")
ax.set_ylabel("Proportion of Latencies")
ax.set_title("Overlap Adjusted Latency of Port Entry to Tone")
# To show the legend
ax.legend()
file_name = "overlap_adjusted_average_latency_plot_{}_date_{}_{}.png".format(FILE_PREFIX, earliest_date, latest_date)

plt.savefig(os.path.join(overlap_adjusted_average_latency_plots_output_directory, file_name))


# Plotting the ratio of overlap adjusted latencies that are less than 10 seconds

In [151]:
less_than_10_second_overlap_adjusted_latency_ratio_output_directory = "./proc/plots/less_than_10_second_overlap_adjusted_latency_ratio/{}_date_{}_{}".format(FILE_PREFIX, earliest_date, latest_date)

In [152]:
less_than_10_second_overlap_adjusted_latency_ratio_output_directory

'./proc/plots/less_than_10_second_overlap_adjusted_latency_ratio/rce_training_date_20240215_20240305'

In [153]:
os.makedirs(less_than_10_second_overlap_adjusted_latency_ratio_output_directory, exist_ok=True)

In [156]:
# # Plotting for each cage
# fig, ax = plt.subplots()
# plt.rcParams["figure.figsize"] = (13.5,7.5)

# # Getting all the rows with the current cage number
# cage_df = grouped_averaged_first_porty_entry_dataframe

# # Plotting a line for each subject
# for subject in cage_df["subject"].unique():
#     # Getting all the rows with the current subject
#     subject_df = cage_df[cage_df["subject"] == subject]
#     # Making the dates into days after the first session by subtracting all the dates by the first date
#     ax.plot(subject_df["date_index"], subject_df["overlap_adjusted_latency_less_than_10_seconds"], '-o', label=subject)

# # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
# ax.set_ylim(0, 1)
# # Labeling the X/Y Axis and the title
# ax.set_xlabel("The Days After the First Session")
# ax.set_ylabel("Proportion of Latencies")
# ax.set_title("Overlap Adjusted Less Than 10sec Latencies from Tone")
# # To show the legend
# ax.legend(loc="lower right")


# file_name = "less_than_10_second_overlap_adjusted_latency_ratio_plot_cage{}_date_{}_{}.png".format(cage, earliest_date, latest_date)
# plt.savefig(os.path.join(less_than_10_second_overlap_adjusted_latency_ratio_output_directory, file_name))


## Plotting the average per strain

## Getting the averages for all the trials in one subject and one day

- We will plot a line for each subject. With the X-axis being the days since the first session. And the Y-axis is the latency values averaged across all the trials for one recording session
- Grouping all the rows with the same subject and date(aka, all the trials in one session). And then getting the mean for each value

In [ ]:
subject_to_cage

In [ ]:
grouped_averaged_first_porty_entry_dataframe["cage"] = grouped_averaged_first_porty_entry_dataframe["subject"].map(subject_to_cage)
grouped_averaged_first_porty_entry_dataframe["strain"] = grouped_averaged_first_porty_entry_dataframe["cage"].map(cage_to_strain)

In [ ]:
strain_averaged_first_porty_entry_dataframe = grouped_averaged_first_porty_entry_dataframe.groupby(by=["strain", "date"]).mean()

In [ ]:
strain_averaged_first_porty_entry_dataframe

In [ ]:
strain_sem_first_porty_entry_dataframe = grouped_averaged_first_porty_entry_dataframe.groupby(by=["strain", "date"]).sem()

## Reformatting the Dataframe for plotting

- Resetting the index so that we can use the data as a value

In [ ]:
strain_averaged_first_porty_entry_dataframe = strain_averaged_first_porty_entry_dataframe.reset_index()
strain_sem_first_porty_entry_dataframe = strain_sem_first_porty_entry_dataframe.reset_index()

In [ ]:
strain_averaged_first_porty_entry_dataframe.head()

In [ ]:
strain_averaged_first_porty_entry_dataframe.groupby(["strain"]).max()

In [ ]:
min_number_of_sessions = strain_averaged_first_porty_entry_dataframe.groupby(["strain"]).max()["date"].min()

In [ ]:
strain_averaged_first_porty_entry_dataframe = strain_averaged_first_porty_entry_dataframe[strain_averaged_first_porty_entry_dataframe["date"] <= min_number_of_sessions]
strain_sem_first_porty_entry_dataframe = strain_sem_first_porty_entry_dataframe[strain_sem_first_porty_entry_dataframe["date"] <= min_number_of_sessions]

In [ ]:
strain_averaged_first_porty_entry_dataframe

In [ ]:
strain_averaged_first_porty_entry_dataframe = strain_averaged_first_porty_entry_dataframe.reset_index(drop=True)
strain_sem_first_porty_entry_dataframe = strain_sem_first_porty_entry_dataframe.reset_index(drop=True)

In [ ]:
strain_sem_first_porty_entry_dataframe

## Plotting the latencies that were adjusted if they greater than 30 seconds

In [ ]:
per_strain_output_directory = "./proc/plots/per_strain_plots/{}_{}_date_{}_{}".format(cohort, FILE_PREFIX, earliest_date, latest_date)

In [ ]:
per_strain_output_directory

In [ ]:
os.makedirs(per_strain_output_directory, exist_ok=True)

In [ ]:
plt.rcParams["figure.figsize"] = (13.5,7.5)

In [ ]:
strain_averaged_first_porty_entry_dataframe.head()

In [ ]:
strain_to_color = {"CD1": "#15616f", "C57": "#ffaf00"}

In [ ]:

fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (13.5,7.5)

# Plotting a line for each subject
for strain in strain_averaged_first_porty_entry_dataframe["strain"].unique():
    # Getting all the rows with the current subject
    strain_df = strain_averaged_first_porty_entry_dataframe[strain_averaged_first_porty_entry_dataframe["strain"] == strain]
    std_df = strain_sem_first_porty_entry_dataframe[strain_sem_first_porty_entry_dataframe["strain"] == strain]

    # Making the dates into days after the first session by subtracting all the dates by the first date
    ax.plot(strain_df["date"], strain_df["latency_adjusted_greater_than_30"], '-o', \
            label=strain, c=strain_to_color[strain])
#     ax.errorbar(strain_df["date"], strain_df["latency_adjusted_greater_than_30"], \
#                 yerr=std_df["latency_adjusted_greater_than_30"], c=strain_to_color[strain])
    ax.fill_between(strain_df["date"], strain_df["latency_adjusted_greater_than_30"]-std_df["latency_adjusted_greater_than_30"], \
                    strain_df["latency_adjusted_greater_than_30"]+std_df["latency_adjusted_greater_than_30"], \
                   alpha=.25, color=strain_to_color[strain])

# Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
ax.set_ylim(0, 30)
# Labeling the X/Y Axis and the title
ax.set_xlabel("The Days After the First Session")
ax.set_ylabel("Adjusted Average Latency of First Entry to Tone Onset")
ax.set_title("Average Latency of Port Entry to Tone Onset for {}".format(cohort))
# To show the legend
ax.legend()
file_name = "strain_average_latency_{}_{}_date_{}_{}.svg".format(cohort, FILE_PREFIX, earliest_date, latest_date)
plt.savefig(os.path.join(per_strain_output_directory, file_name))

In [ ]:

fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (13.5,7.5)

# Plotting a line for each subject
for strain in strain_averaged_first_porty_entry_dataframe["strain"].unique():
    # Getting all the rows with the current subject
    strain_df = strain_averaged_first_porty_entry_dataframe[strain_averaged_first_porty_entry_dataframe["strain"] == strain]
    std_df = strain_sem_first_porty_entry_dataframe[strain_sem_first_porty_entry_dataframe["strain"] == strain]

    # Making the dates into days after the first session by subtracting all the dates by the first date
    ax.plot(strain_df["date"], strain_df["overlap_adjusted_latency_less_than_10_seconds"], '-o', \
            label=strain, c=strain_to_color[strain])
#     ax.errorbar(strain_df["date"], strain_df["overlap_adjusted_latency_less_than_10_seconds"], \
#                 yerr=std_df["overlap_adjusted_latency_less_than_10_seconds"], c=strain_to_color[strain])
    ax.fill_between(strain_df["date"], strain_df["overlap_adjusted_latency_less_than_10_seconds"]-std_df["overlap_adjusted_latency_less_than_10_seconds"], \
                    strain_df["overlap_adjusted_latency_less_than_10_seconds"]+std_df["overlap_adjusted_latency_less_than_10_seconds"], \
                   alpha=.25, color=strain_to_color[strain])

# Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
# ax.set_ylim(0, 30)
# Labeling the X/Y Axis and the title
ax.set_xlabel("The Days After the First Session")
ax.set_ylabel("Proportion of Less than 10 Seconds Latencies")
ax.set_title("Less than 10 Seconds Latency of Port Entry to Tone Onset for {}".format(cohort))
# To show the legend
ax.legend()
file_name = "strain_less_than_10_seconds_latency_{}_{}_date_{}_{}.svg".format(cohort, cage, earliest_date, latest_date)
plt.savefig(os.path.join(per_strain_output_directory, file_name))

In [ ]:
grouped_averaged_first_porty_entry_dataframe["cohort"] = cohort

In [ ]:
grouped_averaged_first_porty_entry_dataframe.head()

In [ ]:
grouped_averaged_first_porty_entry_dataframe_output_directory = "./proc/grouped_averaged_first_porty_entry/{}_{}_date_{}_{}".format(cohort, FILE_PREFIX, earliest_date, latest_date)

In [ ]:
grouped_averaged_first_porty_entry_dataframe_output_directory

In [ ]:
os.makedirs(grouped_averaged_first_porty_entry_dataframe_output_directory, exist_ok=True)

In [ ]:
grouped_averaged_first_porty_entry_dataframe.to_csv(os.path.join(grouped_averaged_first_porty_entry_dataframe_output_directory, "grouped_averaged_first_porty_entry_{}_{}_date_{}_{}.csv".format(cohort, FILE_PREFIX, earliest_date, latest_date)))